### Adaptive Fault Detection with LSTM Models

##### Import libraries needed

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import joblib
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
import pickle


In [2]:
# Data Preparation
# def prepare_data(data, n_steps_in, n_steps_out):
#     X, y = [], []
#     for i in range(len(data)):
#         end_ix = i + n_steps_in
#         out_end_ix = end_ix + n_steps_out - 1
#         if out_end_ix > len(data) - 1:
#             break
#         # seq_x, seq_y = data[i:end_ix, :], data[end_ix-1:out_end_ix, :]
#         seq_x, seq_y = data[i:end_ix, :], data[end_ix:out_end_ix, :]
#         X.append(seq_x)
#         y.append(seq_y)
#     return np.array(X), np.array(y)

def prepare_data(data, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        if out_end_ix > len(data):
            break
        seq_x, seq_y = data[i:end_ix, :], data[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

def generate_random_name(length):
    # random_string = ''.join(random.choices(string.ascii_letters + string.digits, k=length))
    random_string = ''.join(random.choices(string.ascii_letters, k=length))
    # return "expert_models/New_expert_model_" + random_string + ".pkl"
    return "new_lstm_expert_model_" + random_string + ".pkl"



# Load your data
# data = pd.read_csv('your_data.csv').values
# For example, let's create dummy data
# data = np.random.rand(1000, 3)
# file_name = 'waterTank_Golden_reduced.csv'
file_name = 'stuckat0_training_reduced_new.csv'
df = pd.read_csv(file_name, header=0, index_col=0).values


# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(df)

# Prepare data for LSTM
n_steps_in, n_steps_out = 50, 30
X, y = prepare_data(data, n_steps_in, n_steps_out)

# Define the model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps_in, X.shape[2])))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(X.shape[2])))
# model.add(Dense(y.shape[2]))
model.compile(optimizer='adam', loss='mse')

# Fit the model
# model.fit(X, y, epochs=300, verbose=1)
train_history = model.fit(X, y, epochs=10, batch_size=32, verbose=2)

# Evaluate the model
mse = model.evaluate(X, y, verbose=0)
print(mse)

# Save the model and scaler
model.save('expert_models_lstm/lstm_model.h5')
with open('expert_models_lstm/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


Epoch 1/10
623/623 - 34s - loss: 0.0291 - 34s/epoch - 55ms/step
Epoch 2/10
623/623 - 30s - loss: 0.0154 - 30s/epoch - 47ms/step
Epoch 3/10
623/623 - 30s - loss: 0.0143 - 30s/epoch - 48ms/step
Epoch 4/10
623/623 - 30s - loss: 0.0136 - 30s/epoch - 48ms/step
Epoch 5/10
623/623 - 30s - loss: 0.0132 - 30s/epoch - 48ms/step
Epoch 6/10
623/623 - 30s - loss: 0.0130 - 30s/epoch - 47ms/step
Epoch 7/10
623/623 - 30s - loss: 0.0127 - 30s/epoch - 48ms/step
Epoch 8/10
623/623 - 31s - loss: 0.0125 - 31s/epoch - 50ms/step
Epoch 9/10
623/623 - 34s - loss: 0.0123 - 34s/epoch - 54ms/step
Epoch 10/10
623/623 - 30s - loss: 0.0123 - 30s/epoch - 48ms/step
0.013408180326223373


In [28]:
def load_model_and_predict(new_data, n_steps_in, n_steps_out):
    model = load_model('expert_models_lstm/lstm_model.h5')
    with open('expert_models_lstm/scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)

    # Normalize new data
    new_data = scaler.transform(new_data)

    # Prepare input data
    X_new = []
    for i in range(len(new_data) - n_steps_in + 1):
        X_new.append(new_data[i:i + n_steps_in, :])
    X_new = np.array(X_new)

    # Predict
    # y_pred = model.predict(X_new)
    y_pred = model.predict(X_new)
    
    # Inverse transform predictions
    # y_predic = scaler.inverse_transform(y_pred)
    # return y_predic
    return y_pred

# Example of using the function with new data
new_file_name = 'stuckat0_training_reduced_new.csv'
new_data = pd.read_csv(new_file_name, header=0, index_col=0).values


# new_data = np.random.rand(15, 3)
P, q = prepare_data(new_data, n_steps_in, n_steps_out)
# predictions = load_model_and_predict(P, n_steps_in, n_steps_out)
predictions = load_model_and_predict(new_data, n_steps_in, n_steps_out)
print(predictions)


[[[ 1.50678307e-01  2.37167869e-02  9.75447614e-03  4.54175845e-02
    8.20439681e-03]
  [ 1.21236153e-01  6.06210902e-03  9.72293597e-03  2.30790041e-02
    9.77439061e-03]
  [ 9.81832743e-02 -2.68591568e-03  8.84400215e-03  9.86955315e-03
    1.17681213e-02]
  ...
  [ 1.68199435e-01  3.92675027e-03 -2.36986298e-03  1.73154846e-02
    1.18124858e-03]
  [ 1.68046072e-01  3.90850380e-03 -2.37983372e-03  1.73333436e-02
    1.18910521e-03]
  [ 1.67878956e-01  3.87736782e-03 -2.38800328e-03  1.73477083e-02
    1.19851902e-03]]

 [[ 1.51716828e-01  2.36505009e-02  9.67774075e-03  4.56744507e-02
    8.01502727e-03]
  [ 1.22508258e-01  6.00035861e-03  9.51177999e-03  2.36011967e-02
    9.46087018e-03]
  [ 9.95801166e-02 -2.85465643e-03  8.62168241e-03  1.03635825e-02
    1.15375854e-02]
  ...
  [ 1.69664606e-01  3.81430611e-03 -2.34750938e-03  1.73534453e-02
    1.45225599e-03]
  [ 1.69524997e-01  3.80128250e-03 -2.35731993e-03  1.73716955e-02
    1.45931914e-03]
  [ 1.69370472e-01  3.7750490

In [29]:
new_data.shape

(20001, 5)

In [30]:
predictions.shape

(19952, 30, 5)

In [31]:
y.shape

(19922, 30, 5)

#### LSTM Model Training and Saving:

In [ ]:
file_name = 'waterTank_Golden_reduced.csv'
data = pd.read_csv(file_name)  # Replace with your actual data source
values = data['value'].values.reshape(-1, 1)
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_values = scaler.fit_transform(values)



#### Load the Model and Predict New Values:

In [ ]:
# Load the model and scaler
model = load_model('lstm_model.h5')
scaler = joblib.load('scaler.pkl')

# Function to predict the next n values
def predict_next_n_values(data, n=1, look_back=10):
    # Ensure data is scaled using the same scaler
    scaled_data = scaler.transform(data)

    # Prepare the data for prediction
    X = []
    for i in range(len(scaled_data) - look_back):
        X.append(scaled_data[i:(i + look_back), 0])
    X = np.array(X)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    # Predict the next values
    predictions = []
    input_seq = X[-1]
    for _ in range(n):
        pred = model.predict(np.array([input_seq]))[0][0]
        predictions.append(pred)
        input_seq = np.append(input_seq[1:], [[pred]], axis=0)

    # Inverse transform the predictions
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
    return predictions

# Example usage
# Load new data for prediction
new_data = pd.read_csv('new_time_series_data.csv')  # Replace with your actual new data source
new_values = new_data['value'].values.reshape(-1, 1)

# Predict the next 5 values
next_5_values = predict_next_n_values(new_values, n=5)
print(next_5_values)